In [63]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    return conn


if __name__ == '__main__':
    connection = create_connection(r"D:\sources\perso\TripOverview\prototype\data\pythonsqlite.db")

2.6.0


In [39]:
def execute_query(connection, query, data=None):
    cursor = connection.cursor()
    try:
        if data != None:
            cursor.execute(query, data)
        else:
            cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [64]:
create_users_table = """
CREATE TABLE IF NOT EXISTS trip_geo (
  timestamp INTEGER PRIMARY KEY,
  lat FLOAT LATITUDE,
  lon FLOAT LONGITUDE,
  elev FLOAT ELEVATION
);
"""
execute_query(connection, create_users_table)

Query executed successfully


In [43]:
insert_stmt = (
  "INSERT INTO trip_geo (timestamp, lat, lon) "
  "VALUES (?, ?, ?)"
)
data = (1200, 45.0, 35.0)
execute_query(connection, insert_stmt, data)

Query executed successfully


In [65]:
val = [(timestamp, 45.0*timestamp/1600000, 35.0*timestamp/1600000, 1000) for timestamp in range(1700002, 1700002+1000000)]
values = ', '.join(map(str, val))
sql = """INSERT INTO trip_geo (timestamp, lat, lon, elev) VALUES (?, ?, ?, ?) """
cursor = connection.cursor()
cursor.executemany(sql, val)
connection.commit()

In [47]:
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [66]:
select_users = "SELECT * from trip_geo"
users = execute_read_query(connection, select_users)
print(len(users))

1000000


In [62]:
connection.close()

In [71]:
import pandas as pd
sql_query = pd.read_sql_query('''SELECT * from trip_geo''', connection)
sql_query

,timestamp,lat,lon,elev
0,1700002,47.812556,37.187544,1000.0
1,1700003,47.812584,37.187566,1000.0
2,1700004,47.812613,37.187587,1000.0
3,1700005,47.812641,37.187609,1000.0
4,1700006,47.812669,37.187631,1000.0
...,...,...,...,...
999995,2699997,75.937416,59.062434,1000.0
999996,2699998,75.937444,59.062456,1000.0
999997,2699999,75.937472,59.062478,1000.0
999998,2700000,75.937500,59.062500,1000.0


In [85]:
import geojson
features = []
insert_features = lambda x: features.append(
    geojson.Feature(geometry=geojson.LineString((x["lat"],
                                                 x["lon"],
                                                 x["elev"])),
                    properties=dict(times=x["timestamp"], 'style': {'color' : row['color']},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': row['color'],
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': 7
                })))

sql_query.apply(insert_features, axis=1)

print(geojson.FeatureCollection(features))
with open('map1.geojson', 'w', encoding='utf8') as fp:
    geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [80]:
geojson.utils.generate_random("LineString")

{"coordinates": [[129.26215, 39.007409], [154.972965, -6.694012], [91.402371, -22.885289]], "type": "LineString"}

In [ ]:
import folium

m = folium.Map([48.2, 16.4], zoom_start=10)
 
folium.Choropleth(
    graph[graph.geometry.length>0.001],
    line_weight=3,
    line_color='blue'
).add_to(m)
 
m